In [8]:
## Full backend analysis for the CREEDS and L1000 dataset 
import os
import pandas as pd
from pandas.compat import StringIO
import numpy as np
from numpy import loadtxt
import sys
import json
from pprint import pprint
import objectpath
import csv
import re
import matplotlib.pyplot as plt
import json, requests
from pprint import pprint
import itertools
import scipy
from scipy.spatial import distance
from sklearn.metrics.pairwise import pairwise_distances
from clustergrammer_widget import *
def get_geneset(df, indexer):
    df_ = df.loc[indexer, :]
    return list(df_[df_ == 1].index)

In [38]:
### CREEDS DISEASE CARD PRELOAD
with open("L1000_CREEDS_Similar_Drug_output.txt", "r") as fd:
    similar_L1000_CREEDS_drugs = fd.read().splitlines()
    
# CREEDS up and down genes for disease signatures
with open("disease_signatures-v1.0.json") as f:
    CREEDS_data = json.load(f)
CREEDS_df = pd.DataFrame(CREEDS_data)

# generate the up and down gene signatures
CREEDS_up_genes = {
    row['id']: row['up_genes']
    for row in CREEDS_data
}
CREEDS_down_genes = {
    row['id']: row['down_genes']
    for row in CREEDS_data
}

### CREEDS Disease cards

DOI = "psoriasis"
CREEDS_disease = CREEDS_df[CREEDS_df["disease_name"] == DOI]
CREEDS_disease_ids = list(CREEDS_disease["id"]) # all the signatures to print out

for a in CREEDS_disease_ids:
    CREEDS_up_gene_sigs = pd.DataFrame(CREEDS_up_genes[a])
    filename1 = (str(a) + "_CREEDS_disease_sig_up_genes.csv")
    CREEDS_up_gene_sigs.to_csv(filename1) # csv for up genes
    #print(filename1)
    
    CREEDS_down_gene_sigs = pd.DataFrame(CREEDS_down_genes[a])
    filename2 = (str(a) + "_CREEDS_disease_sig_down_genes.csv")
    CREEDS_down_gene_sigs.to_csv(filename2) # csv for down genes


In [12]:

### CREEDS DRUG CARD 


DrOI = "digoxin"


CREEDS_URL = 'http://amp.pharm.mssm.edu/CREEDS/'
CREEEDS_Drug_response = requests.get(CREEDS_URL + 'search', params={'q':DrOI})
if CREEEDS_Drug_response.status_code == 200:
    CREEDS_drug_output_df = pd.DataFrame(CREEEDS_Drug_response.json())
    CREEDS_drug_output_ids = list(CREEDS_drug_output_df["id"])
    
    for a in CREEDS_drug_output_ids:
        CREEDS_URL = 'http://amp.pharm.mssm.edu/CREEDS/'
        CREEDS_drug_sigs_response = requests.get(CREEDS_URL + 'api', params={'id':'drug:DM609'})
        if CREEDS_drug_sigs_response.status_code == 200:
            CREEDS_drug_sigs_response_json = CREEDS_drug_sigs_response.json()
            
            ## up genes
            CREEDS_drug_sigs_up_genes = CREEDS_drug_sigs_response_json['up_genes']
            CREEDS_drug_sigs_up_genes_df = pd.DataFrame(CREEDS_drug_sigs_up_genes) # this is the up genes dataframe
            filename1 = (a + "_CREEDS_drug_sig_up_genes.csv")
            #CREEDS_drug_sigs_up_genes_df.to_csv(filename1) # this saves the df as a csv
            
            ## down genes
            CREEDS_drug_sigs_down_genes = CREEDS_drug_sigs_response_json['down_genes']
            CREEDS_drug_sigs_down_genes_df = pd.DataFrame(CREEDS_drug_sigs_down_genes)# this is the down genes dataframe
            filename2 = (a + "_CREEDS_drug_sig_down_genes.csv")
            #CREEDS_drug_sigs_down_genes_df.to_csv(filename2)
            #CREEDS_drug_sigs_down_genes = CREEDS_drug_sigs_response_json['down_genes'] # this saves the df as a csv
            print(filename2)
            
            ## json propagation
            #pprint(response.json())
            #json.dump(response.json(), open(a + '_CREEDS_Drug_sig.json', 'w'), indent=4) # if the user wants the entire json, they can download this
            

drug:DM609_CREEDS_drug_sig_down_genes.csv
drug:DM610_CREEDS_drug_sig_down_genes.csv
drug:DM611_CREEDS_drug_sig_down_genes.csv
drug:DM2490_CREEDS_drug_sig_down_genes.csv
drug:DM2491_CREEDS_drug_sig_down_genes.csv
drug:DM2492_CREEDS_drug_sig_down_genes.csv
drug:DM2493_CREEDS_drug_sig_down_genes.csv
drug:DM2494_CREEDS_drug_sig_down_genes.csv
drug:DM2495_CREEDS_drug_sig_down_genes.csv
drug:DM2496_CREEDS_drug_sig_down_genes.csv


In [16]:
CREEDS_df

,cell_type,ctrl_ids,curator,disease_name,do_id,down_genes,geo_id,id,organism,pert_ids,platform,umls_cui,up_genes,version
0,Muscle - Striated (Skeletal) (MMHCC),"[GSM4372, GSM4373, GSM4374, GSM4375, GSM4376]",Joel.Dudley,Duchenne muscular dystrophy,DOID:11723,"[[Rpl41, -0.1526295691728592], [Tnnt3, -0.1450...",GSE466,dz:328,mouse,"[GSM4377, GSM4378, GSM4379, GSM4380, GSM4381]",GPL81,C0013264,"[[Actc1, 0.1277850866317749], [Lyz1, 0.1037828...",1.0
1,Ganglioneuroblastoma,"[GSM282582, GSM282583, GSM282587, GSM282588, G...",Joel.Dudley,Nicotine addiction,None,"[[BEX1, -0.08722281455993652], [ZNF423, -0.077...",GSE11208,dz:325,human,"[GSM282584, GSM282585, GSM282586, GSM282590, G...",GPL570,C0028043,"[[RPS18, 0.19893527030944824], [COX2, 0.193066...",1.0
2,Nose,"[GSM286649, GSM286650, GSM286651, GSM286655, G...",Joel.Dudley,Rhinovirus infection,None,"[[HLA-DRB4, -0.2605530321598053], [BPIFB1, -0....",GSE11348,dz:324,human,"[GSM286646, GSM286647, GSM286648, GSM286652, G...",GPL570,C0276447,"[[FCGBP, 0.1933731883764267], [S100A4, 0.19266...",1.0
3,Myocardial tissue,"[GSM82393, GSM82394, GSM82395, GSM82396, GSM82...",Joel.Dudley,dilated cardiomyopathy,DOID:12930,"[[CKM, -0.20369680225849152], [SMPX, -0.177489...",GSE3586,dz:323,human,"[GSM82408, GSM82409, GSM82410, GSM82411, GSM82...",GPL3050,C0007193,"[[MB, 0.3557915985584259], [MYL2, 0.2121689766...",1.0
4,Peripheral blood mononuclear cell,"[GSM66671, GSM66672, GSM66673, GSM66674, GSM66...",Joel.Dudley,bacterial infectious disease,DOID:104,"[[RPS27, -0.20674751698970795], [CD74, -0.1724...",GSE3026,dz:322,human,"[GSM66617, GSM66618, GSM66619, GSM66620, GSM66...",GPL8300,C0004623,"[[HLA-DRB5, 0.26361608505249023], [IGHA2, 0.16...",1.0
5,Lung Tissue,"[GSM533882, GSM533883, GSM533884, GSM533885, G...",Joel.Dudley,idiopathic interstitial pneumonia,DOID:2797,"[[FOS, -0.23119544982910156], [FOSB, -0.228891...",GSE21369,dz:321,human,"[GSM533888, GSM533889, GSM533890, GSM533891, G...",GPL570,C0085786,"[[IGHA2, 0.26548025012016296], [SFTPC, 0.15447...",1.0
6,Renal Tissue,"[GSM9920, GSM9921, GSM9922, GSM9923, GSM9924, ...",Joel.Dudley,type 2 diabetes mellitus,DOID:9352,"[[Acy3, -0.2007710188627243], [Cyp2e1, -0.1731...",GSE642,dz:320,mouse,"[GSM9926, GSM9927, GSM9928, GSM9929, GSM9930, ...",GPL81,C0011860,"[[Mt1, 0.14427176117897034], [Gsta2, 0.1420447...",1.0
7,Back Skin (K5-Stat3C psoriasis model),"[GSM684702, GSM684703]",cadimo,psoriasis,DOID:8893,"[[Krt77, -0.1783602088689804], [Krtap8-1, -0.1...",GSE27628,dz:821,mouse,"[GSM684700, GSM684701]",GPL1261,C0033860,"[[S100a8, 0.2103414386510849], [S100a9, 0.2005...",1.0
8,Back Skin (K5-TGFbeta1 psoriasis model),"[GSM684707, GSM684708, GSM684709]",cadimo,psoriasis,DOID:8893,"[[Sprr2a2, -0.1362936645746231], [Stfa3, -0.11...",GSE27628,dz:822,mouse,"[GSM684704, GSM684705, GSM684706]",GPL1261,C0033860,"[[Pvalb, 0.1734006106853485], [Atp2a1, 0.16719...",1.0
9,Back Skin (K5-Tie2 psoriasis model),"[GSM684679, GSM684680, GSM684681]",cadimo,psoriasis,DOID:8893,"[[Krt15, -0.1630019098520279], [Acta1, -0.1402...",GSE27628,dz:823,mouse,"[GSM684676, GSM684677, GSM684678]",GPL1261,C0033860,"[[Krt1, 0.30147773027420044], [Calm4, 0.208382...",1.0


In [41]:
CREEDS_disease_ids

['dz:326', 'dz:117', 'dz:950', 'dz:951', 'dz:949', 'dz:948']

In [46]:
for a in CREEDS_disease_ids:
    CREEDS_down_gene_sigs = pd.DataFrame(CREEDS_down_genes[a])
    filename2 = (str(a) + "_CREEDS_disease_sig_down_genes.csv")
    CREEDS_down_gene_sigs.to_csv(filename2)

In [47]:
CREEDS_down_gene_sigs

,0,1
0,SLCO2B1,-0.171130
1,CXCL10,-0.120015
2,TBC1D8B,-0.109493
3,WNT16,-0.104169
4,SAMD9L,-0.098908
5,KHDRBS2,-0.098364
6,IFIT1,-0.093798
7,IFI44L,-0.090453
8,FAM198B,-0.075503
9,ADRBK2,-0.074995
